In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torch

from torch.utils.data import DataLoader
from torchsummary import summary
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.functional import F

from data_process.misclassified_data import *
from data_process.getdata import *

from model.model import *
from model.ghostnet_model import *
from model.model_test import *
from model.model_train import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.chdir('d:\Python Projects\EVA')
cwd = os.getcwd()

ModuleNotFoundError: No module named 'data_process'

In [2]:
from nikshriai import model

ModuleNotFoundError: No module named 'nikshriai'

In [3]:
model_dir = os.path.join(cwd, 'Assignment-6/saved_models')
img_dir = os.path.join(cwd, 'Assignment-6/saved_images')
print(model_dir, img_dir)

NameError: name 'cwd' is not defined

In [4]:
train = GetTrainData.download_train_data(cwd)
test = GetTrainData.download_test_data(cwd)

NameError: name 'GetTrainData' is not defined

In [5]:
#
SEED = 1
device = 'cuda' if torch.cuda else 'cpu'

# for Reproducable results
torch.manual_seed(SEED)

#Setting seed if CUDA
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# The Model

In [6]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

NameError: name 'Net' is not defined

# GhostModel

In [7]:
model_ghost = GhostNet().to(device)

NameError: name 'GhostNet' is not defined

In [8]:
# Assignment 6

def get_optim(mode):

    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.06)

    if mode is 'l2_bn':
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0, dampening=0, weight_decay=0.00001, nesterov=False)
        model_training(model, device, train_dataloader, optimizer, train_acc, train_losses)
        scheduler.step()
    
    elif mode is 'l1_bn':
        model_training(model, device, train_dataloader, optimizer, train_acc, train_losses, l1_loss=True)
        scheduler.step()

    elif mode is 'l1_l2_bn':
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0, dampening=0, weight_decay=0.00001, nesterov=False)
        model_training(model, device, train_dataloader, optimizer, train_acc, train_losses, l1_loss=True)
        scheduler.step()

    elif mode is 'gbn':
        model_training(model, device, train_dataloader, optimizer, train_acc, train_losses)
        scheduler.step()

    elif mode is 'l1_l2_gbn':
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0, dampening=0, weight_decay=0.00001, nesterov=False)
        model_training(model, device, train_dataloader, optimizer, train_acc, train_losses, l1_loss=True)
        scheduler.step()

In [9]:
import gc
EPOCHS = 15

# list_modes = ['gbn', 'l1_l2_gbn', 'l1_bn', 'l2_bn', 'l1_l2_bn']
list_modes = ['gbn', 'l1_l2_gbn', 'l1_bn', 'l2_bn', 'l1_l2_bn']
fig, axs = plt.subplots(2,2, figsize=(25,20))

axs[0,0].set_title('Train Losses')
axs[0,1].set_title('Training Accuracy')
axs[1,0].set_title('Test Losses')
axs[1,1].set_title('Test Accuracy')


for mode in list_modes:
    gc.collect()
    print(f'Mode is {mode}')

    if mode in ['gbn', 'l1_l2_gbn']: # Larger batch sizes and GhostNet model for GBN
        model = GhostNet().to(device)
        dataloader_args = dict(shuffle=True, num_workers=4, pin_memory=True, batch_size=256) if torch.cuda.is_available() else dict(shuffle=True, batch_size=64) 
    
    else:
        model = Net().to(device) # Normal batch-sizes for rest
        dataloader_args = dict(shuffle=True, num_workers=4, pin_memory=True, batch_size=128) if torch.cuda.is_available() else dict(shuffle=True, batch_size=64)

    test_acc = []
    test_losses = []
    train_acc = []
    train_losses = []
    misclassified = None

    train_dataloader = DataLoader(train,**dataloader_args)
    test_dataloader = DataLoader(test, **dataloader_args)
    for epoch in range(EPOCHS):
        print('EPOCH - ', epoch)
        get_optim(mode)
        misclassified = model_testing(model, device, test_dataloader, test_acc, test_losses)
    
    # Plotting the misclassified images for GBN Model
    if mode is 'gbn':
        plot_misclassified(misclassified)

    axs[0,0].plot(train_losses)
    axs[0,1].plot(train_acc)
    axs[1,0].plot(test_losses)
    axs[1,1].plot(test_acc)

axs[0,0].legend(list_modes)
axs[0,1].legend(list_modes)
axs[1,0].legend(list_modes)
axs[1,1].legend(list_modes)
fig.savefig(os.path.join(img_dir, 'metrics.png'))


Mode is gbn


NameError: name 'GhostNet' is not defined

In [10]:
fig.savefig(os.path.join('metrics.png'))